This notebook is adapted from: https://github.com/pytorch/examples/tree/master/vae which uses ReLUs and the adam optimizer, instead of sigmoids and adagrad as in the original paper. This gives faster convergence.

The notebook is set up to receive images that are 28x28 pixels in size and encode them into 2 latent variables. It uses the MNIST dataset by default. If you don't have it already the notebook will automatically download it for you.

You need to create a directory called "results" in the same directory that you run the notebook from. 

The outputs are: 

* **reconstruction_NN.png** - the top row is a random selection of 8 samples from the test dataset and the lower row is the reconstruction using the model based on their auto-encoded latent variables.

* **sample_NN.png** - this is a selection of 64 randomly created samples based on a random selection of latent variable values from the model.

* **manifold.png** - this is the equivalent of Fig4(b) in Kingma & Welling.

---

In [1]:
from __future__ import print_function
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
from torch.distributions import Normal
from torchsummary import summary
from VAE import VAE

First set the parameters of the training:

In [2]:
batch_size=128
epochs=10
no_cuda=True
seed=1
log_interval=100

Decide whether you can use a GPU:

In [3]:
cuda = no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if cuda else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}

Set a random seed:

In [4]:
torch.manual_seed(seed)

Load the training data:

In [5]:
train_loader = torch.utils.data.DataLoader(
                            datasets.MNIST('../data', train=True, download=True,
                            transform=transforms.ToTensor()),
                            batch_size=batch_size, shuffle=True, **kwargs)

Load the test data:

In [6]:
test_loader = torch.utils.data.DataLoader(
                            datasets.MNIST('../data', train=False, 
                            transform=transforms.ToTensor()),
                            batch_size=batch_size, shuffle=True, **kwargs)

Call the network:

In [7]:
model = VAE().to(device)

Have a look at the different layers:

In [8]:
summary(model,(1,28,28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                  [-1, 400]         314,000
            Linear-2                    [-1, 2]             802
            Linear-3                    [-1, 2]             802
            Linear-4                  [-1, 400]           1,200
            Linear-5                  [-1, 784]         314,384
Total params: 631,188
Trainable params: 631,188
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 2.41
Estimated Total Size (MB): 2.42
----------------------------------------------------------------


Define the optimizer and the learning rate:

In [9]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)

Implement the loss function from the Kingma & Welling paper (Appendix B):

In [10]:
def loss_function(recon_x, x, mu, logvar):
    
    BCE = F.binary_cross_entropy(recon_x, x.view(-1,784), reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + KLD

Define a function for training:

In [11]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))

Define a function for testing:

In [12]:
def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (data, _) in enumerate(test_loader):
            data = data.to(device)
            recon_batch, mu, logvar = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar).item()
            if i == 0:
                n = min(data.size(0), 8)
                comparison = torch.cat([data[:n],
                                      recon_batch.view(batch_size, 1, 28, 28)[:n]])
                save_image(comparison.cpu(),
                         'results/reconstruction_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

Loop over epochs, updating the training each time and testing the updated model against the test dataset:

In [13]:
for epoch in range(1, epochs + 1):
        train(epoch)
        test(epoch)
        with torch.no_grad():
            sample = torch.randn(64, 2).to(device)
            sample = model.decode(sample).cpu()
            save_image(sample.view(64, 1, 28, 28),
                       'results/sample_' + str(epoch) + '.png')

Train Epoch: 1 [0/60000 (0%)]	Loss: 552.574097
Train Epoch: 1 [12800/60000 (21%)]	Loss: 190.118073
Train Epoch: 1 [25600/60000 (43%)]	Loss: 182.392120
Train Epoch: 1 [38400/60000 (64%)]	Loss: 177.693863
Train Epoch: 1 [51200/60000 (85%)]	Loss: 173.205551
====> Epoch: 1 Average loss: 191.6132
====> Test set loss: 172.1348
Train Epoch: 2 [0/60000 (0%)]	Loss: 174.518036
Train Epoch: 2 [12800/60000 (21%)]	Loss: 163.312073
Train Epoch: 2 [25600/60000 (43%)]	Loss: 169.693390
Train Epoch: 2 [38400/60000 (64%)]	Loss: 165.049606
Train Epoch: 2 [51200/60000 (85%)]	Loss: 164.099014
====> Epoch: 2 Average loss: 168.1606
====> Test set loss: 164.8987
Train Epoch: 3 [0/60000 (0%)]	Loss: 166.214996
Train Epoch: 3 [12800/60000 (21%)]	Loss: 165.291962
Train Epoch: 3 [25600/60000 (43%)]	Loss: 162.137985
Train Epoch: 3 [38400/60000 (64%)]	Loss: 162.485703
Train Epoch: 3 [51200/60000 (85%)]	Loss: 161.068802
====> Epoch: 3 Average loss: 163.3929
====> Test set loss: 161.6210
Train Epoch: 4 [0/60000 (0%)]	L

Recreate Figure 4(b) from Kingma & Welling:

In [14]:
nside = 20
x, y = torch.meshgrid([torch.linspace(0.,1.,nside), torch.linspace(0.,1.,nside)])
m = Normal(torch.tensor([0.0]), torch.tensor([1.0]))
x = m.icdf(x).view(-1,nside**2)
y = m.icdf(y).view(-1,nside**2)
z = torch.cat((x,y),0).t()
with torch.no_grad():
    sample = z.to(device)
    sample = model.decode(sample).cpu()
    save_image(sample.view(nside**2, 1, 28, 28),
                   'results/manifold.png', nrow=nside)